In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
# nltk.download('punkt')
import re
from bs4 import BeautifulSoup
from sklearn.decomposition import TruncatedSVD
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('Twitter.csv')
df.head()

,clean_text,category,category_sentiment
0,when modi promised “minimum government maximum...,-1,negative
1,talk all the nonsense and continue all the dra...,0,neutral
2,what did just say vote for modi welcome bjp t...,1,positive
3,asking his supporters prefix chowkidar their n...,1,positive
4,answer who among these the most powerful world...,1,positive


In [3]:
df=df.dropna()
df['category'] = df['category'].astype(int)
df = df.reset_index(drop=True)
df['category'].value_counts()

 1    72254
 0    62713
-1    43019
Name: category, dtype: int64

In [4]:
# df=df[df['category']!=0]
df['category'].value_counts()

 1    72254
 0    62713
-1    43019
Name: category, dtype: int64

In [5]:
df = df.sample(frac=1).reset_index(drop=True) #shuffling of tweets
data=df[df['category']==-1][:43019]
data=data.append(df[df['category']==0][:43019])
data=data.append(df[df['category']==1][:43019])
data = data.reset_index(drop=True)
display(data['category'].value_counts())
data

C:\Users\CPORGALINAS\AppData\Local\Temp\ipykernel_14288\490375356.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['category']==0][:43019])
C:\Users\CPORGALINAS\AppData\Local\Temp\ipykernel_14288\490375356.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['category']==1][:43019])


-1    43019
 0    43019
 1    43019
Name: category, dtype: int64

,clean_text,category,category_sentiment
0,\r\ntalking the strict laws crack down heavily...,-1,negative
1,modi never made such promisehe had said the en...,-1,negative
2,when stupid ppl like can give its due credit n...,-1,negative
3,not alone evks once asked what amma and modi w...,-1,negative
4,rahul thief modi chowkidar wait and watch only...,-1,negative
...,...,...,...
129052,why fear modi ruling koi hijra nahi hai sher h...,1,positive
129053,the sceintists would have announced one would ...,1,positive
129054,for winning elections\r\nmodi worship god\r\na...,1,positive
129055,then why modi better his speeches and mocking ...,1,positive


In [6]:
data['pre_process'] = data['clean_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

C:\Users\CPORGALINAS\Documents\Machine Learning\Revalida 2\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

In [8]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [33]:
def Negation(sentence):	
    '''
    Input: Tokenized sentence (List of words)
    Output: Tokenized sentence with negation handled (List of words)
    '''
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    sentence = ' '.join(sentence)
    return sentence

In [10]:
# data['pre_process']=data['pre_process'].apply(lambda x:Negation(nltk.word_tokenize(x)))

In [11]:
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [12]:
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [13]:
data.head()

,clean_text,category,category_sentiment,pre_process
0,\r\ntalking the strict laws crack down heavily...,-1,negative,talking the strict laws crack down heavily the...
1,modi never made such promisehe had said the en...,-1,negative,modi never made such promisehe had said the en...
2,when stupid ppl like can give its due credit n...,-1,negative,when stupid ppl like can give its due credit n...
3,not alone evks once asked what amma and modi w...,-1,negative,not alone evks once asked what amma and modi w...
4,rahul thief modi chowkidar wait and watch only...,-1,negative,rahul thief modi chowkidar wait and watch only...


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['pre_process'], data['category'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (96792,) (96792,) Test:  ((32265,), (32265,))


In [15]:
# print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer= TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [16]:
# Training a Linear SVC
clf = LinearSVC(random_state=0)
clf.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [17]:
# Predict on the test set
y_pred=clf.predict(X_test)

In [18]:
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

Accuracy:  0.9167828916782892


In [19]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.91      0.89      0.90     10740
           0       0.92      0.95      0.94     10764
           1       0.92      0.91      0.91     10761

    accuracy                           0.92     32265
   macro avg       0.92      0.92      0.92     32265
weighted avg       0.92      0.92      0.92     32265



In [35]:
# Input your own tweet
tweet = "I like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()
tweet = contractions(tweet)
tweet = Negation(nltk.word_tokenize(tweet))

print(tweet)
# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

I like apples .
Neutral


In [36]:
# Input your own tweet
tweet = "I do not like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()
tweet = contractions(tweet)
tweet = Negation(nltk.word_tokenize(tweet))

print(tweet)

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

I do dislike apples .
Neutral


In [38]:
# Input your own tweet
tweet = "I am happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()
tweet = contractions(tweet)
tweet = Negation(nltk.word_tokenize(tweet))

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

Positive


In [40]:
# Input your own tweet
tweet = "I am not happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()
tweet = contractions(tweet)
tweet = Negation(nltk.word_tokenize(tweet))

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

Negative


In [ ]:
# # Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

In [ ]:
# # Predict on the test set
y_pred=clf.predict(X_test)

In [ ]:
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

In [ ]:
print(classification_report(Y_test, y_pred))

In [ ]:
# Input your own tweet
tweet = "I like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I do not like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I am happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I am not happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Training for Logistic Regression
clf = LogisticRegression(max_iter=1000,solver='saga')
clf.fit(X_train, Y_train)

In [ ]:
# Predict on the test set
y_pred=clf.predict(X_test)
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

In [ ]:
print(classification_report(Y_test, y_pred))

In [ ]:
# Input your own tweet
tweet = "I like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I do not like apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I am happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [ ]:
# Input your own tweet
tweet = "I am not happy."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

In [21]:
from joblib import dump, load

model_file = "linearSVC.joblib"

dump(clf, model_file)

['linearSVC.joblib']